In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: c:\Users\lulev\Desktop\KTH\mlfs-book
Added the following directory to the PYTHONPATH: c:\Users\lulev\Desktop\KTH\mlfs-book
HopsworksSettings initialized!


### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

---

In [3]:
project = hopsworks.login(engine="python")

2025-11-15 11:53:19,193 INFO: Initializing external client
2025-11-15 11:53:19,194 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-15 11:53:20,956 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279155


## <span style='color:#ff5f27'> Load parameters of all cities </span>

In [4]:
import csv

# Load all parameters for each city
def load_city(csv_path):
    city_dict = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            city = row["city"].strip().lower()
            url = row["url"].strip()
            latitude = row["latitude"].strip()
            longitude = row["longitude"].strip()
            # Can use the below line instead if long & lat is unknown, but I got blocked from the ost API :(
            # latitude, longitude = util.get_city_coordinates(city)
            csv_file=f"{root_dir}/data/{city}.csv"
            city_dict[city] = {
                "url": url,
                "csv_file": csv_file,
                "latitude": float(latitude),
                "longitude": float(longitude)
            }
    return city_dict

city_dict = load_city(f"{root_dir}/data/urls.csv")

## <span style='color:#ff5f27'> Get AQICN API KEY </span>

In [5]:
today = datetime.date.today()
# taken from ~/.env. You can also replace settings.AQICN_API_KEY with the api key value as a string "...."
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)
AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
# Replace any existing secret with the new value
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
    print("Replacing existing AQICN_API_KEY")

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Found AQICN_API_KEY: 4b11809c8f7e83cc9bfe296a4324433b0a0055ba
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

### <span style='color:#ff5f27'> The parameters of all cities currently in use </span>

In [6]:
city_dict

{'bodo': {'url': 'https://api.waqi.info/feed/@12706',
  'csv_file': 'c:\\Users\\lulev\\Desktop\\KTH\\mlfs-book/data/bodo.csv',
  'latitude': 67.28,
  'longitude': 14.38},
 'moirana': {'url': 'https://api.waqi.info/feed/@12698',
  'csv_file': 'c:\\Users\\lulev\\Desktop\\KTH\\mlfs-book/data/moirana.csv',
  'latitude': 66.33,
  'longitude': 14.18}}

In [7]:
# try:
#     aq_today_df = util.get_pm25(aqicn_url, city, today, AQICN_API_KEY)
# except hopsworks.RestAPIError:
#     print("It looks like the AQICN_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

# aq_today_df.head()

## <span style='color:#ff5f27'> Read CSV file into a DataFrame </span>

The cell below will read up historical air quality data as a CSV file into a Pandas DataFrame

In [8]:
df_list = []
for city in city_dict:  
    df_temp = pd.read_csv(city_dict[city]["csv_file"],  parse_dates=['date'], skipinitialspace=True)
    df_temp["city"] = city
    df_temp["url"] = city_dict[city]["url"]
    df_list.append(df_temp)
df = pd.concat(df_list, ignore_index=True)
df

,date,pm25,pm10,city,url
0,2025-11-01,10.0,4.0,bodo,https://api.waqi.info/feed/@12706
1,2025-11-02,4.0,11.0,bodo,https://api.waqi.info/feed/@12706
2,2025-11-03,8.0,6.0,bodo,https://api.waqi.info/feed/@12706
3,2025-11-04,6.0,9.0,bodo,https://api.waqi.info/feed/@12706
4,2025-11-05,21.0,4.0,bodo,https://api.waqi.info/feed/@12706
...,...,...,...,...,...
4223,2020-01-04,NaN,8.0,moirana,https://api.waqi.info/feed/@12698
4224,2020-01-05,NaN,8.0,moirana,https://api.waqi.info/feed/@12698
4225,2020-01-06,NaN,11.0,moirana,https://api.waqi.info/feed/@12698
4226,2020-01-07,NaN,10.0,moirana,https://api.waqi.info/feed/@12698


## <span style='color:#ff5f27'>Data cleaning</span>


### Rename columns if needed and drop unneccessary columns

We want to have a DataFrame with 2 columns - `date`, `pm25`, `city`, `url` after this cell below:

## Check the data types for the columns in your DataFrame

 * `date` should be of type   datetime64[ns] 
 * `pm25` should be of type float64

In [9]:
df_aq = df[['date', 'pm25', 'city', 'url']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')

df_aq

,date,pm25,city,url
0,2025-11-01,10.0,bodo,https://api.waqi.info/feed/@12706
1,2025-11-02,4.0,bodo,https://api.waqi.info/feed/@12706
2,2025-11-03,8.0,bodo,https://api.waqi.info/feed/@12706
3,2025-11-04,6.0,bodo,https://api.waqi.info/feed/@12706
4,2025-11-05,21.0,bodo,https://api.waqi.info/feed/@12706
...,...,...,...,...
4223,2020-01-04,NaN,moirana,https://api.waqi.info/feed/@12698
4224,2020-01-05,NaN,moirana,https://api.waqi.info/feed/@12698
4225,2020-01-06,NaN,moirana,https://api.waqi.info/feed/@12698
4226,2020-01-07,NaN,moirana,https://api.waqi.info/feed/@12698


In [10]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4228 entries, 0 to 4227
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4228 non-null   datetime64[ns]
 1   pm25    4208 non-null   float32       
 2   city    4228 non-null   object        
 3   url     4228 non-null   object        
dtypes: datetime64[ns](1), float32(1), object(2)
memory usage: 115.7+ KB


## <span style='color:#ff5f27'>Drop any rows with missing data </span>
It will make the model training easier if there is no missing data in the rows, so we drop any rows with missing data.

In [11]:
df_aq.dropna(inplace=True)
# df_aq["lag1"] = df_aq["pm25"].shift(1)
# df_aq["lag2"] = df_aq["pm25"].shift(2)
# df_aq["lag3"] = df_aq["pm25"].shift(3)
# df_aq["rolling"] = (df_aq.set_index("date")["pm25"].rolling(window=3).mean().reset_index(drop=True))
df_aq

,date,pm25,city,url
0,2025-11-01,10.0,bodo,https://api.waqi.info/feed/@12706
1,2025-11-02,4.0,bodo,https://api.waqi.info/feed/@12706
2,2025-11-03,8.0,bodo,https://api.waqi.info/feed/@12706
3,2025-11-04,6.0,bodo,https://api.waqi.info/feed/@12706
4,2025-11-05,21.0,bodo,https://api.waqi.info/feed/@12706
...,...,...,...,...
4207,2020-03-27,11.0,moirana,https://api.waqi.info/feed/@12698
4208,2020-03-28,11.0,moirana,https://api.waqi.info/feed/@12698
4209,2020-03-29,9.0,moirana,https://api.waqi.info/feed/@12698
4210,2020-03-30,21.0,moirana,https://api.waqi.info/feed/@12698


In [12]:
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4208 entries, 0 to 4211
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4208 non-null   datetime64[ns]
 1   pm25    4208 non-null   float32       
 2   city    4208 non-null   object        
 3   url     4208 non-null   object        
dtypes: datetime64[ns](1), float32(1), object(2)
memory usage: 147.9+ KB


---

## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

## <span style='color:#ff5f27'>Download the Historical Weather Data </span>

https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant

We will download the historical weather data for your `city` by first extracting the earliest date from your DataFrame containing the historical air quality measurements.

We will download all daily historical weather data measurements for your `city` from the earliest date in your air quality measurement DataFrame. It doesn't matter if there are missing days of air quality measurements. We can store all of the daily weather measurements, and when we build our training dataset, we will join up the air quality measurements for a given day to its weather features for that day. 

The weather features we will download are:

 * `temperature (average over the day)`
 * `precipitation (the total over the day)`
 * `wind speed (average over the day)`
 * `wind direction (the most dominant direction over the day)`


In [13]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df_list = []
for city in city_dict:
    temp_weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), city_dict[city]["latitude"], city_dict[city]["longitude"])
    weather_df_list.append(temp_weather_df)
weather_df = pd.concat(weather_df_list, ignore_index=True)

weather_df.loc[weather_df["city"] == "selfors", "city"] = "moirana" # Set the right city key since we used a different part of moirana for the weather

Coordinates 67.31107330322266°N 14.587156295776367°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 66.32688903808594°N 13.988268852233887°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [14]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4292 entries, 0 to 4291
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         4292 non-null   datetime64[ns]
 1   temperature_2m_mean          4292 non-null   float32       
 2   precipitation_sum            4292 non-null   float32       
 3   wind_speed_10m_max           4292 non-null   float32       
 4   wind_direction_10m_dominant  4292 non-null   float32       
 5   city                         4292 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 134.2+ KB


## <span style='color:#ff5f27'>Define Data Validation Rules </span>

We will validate the air quality measurements (`pm25` values) before we write them to Hopsworks.

We define a data validation rule (an expectation in Great Expectations) that ensures that `pm25` values are not negative or above the max value available by the sensor.

We will attach this expectation to the air quality feature group, so that we validate the `pm25` data every time we write a DataFrame to the feature group. We want to prevent garbage-in, garbage-out.

In [15]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

## Expectations for Weather Data
Here, we define an expectation for 2 columns in our weather DataFrame - `precipitation_sum` and `wind_speed_10m_max`, where we expect both values to be greater than zero, but less than 1000.

In [16]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

---

### <span style="color:#ff5f27;"> 🔮 STEP 11: Connect to Hopsworks and save the sensor country, city, street names as a secret</span>

In [17]:
fs = project.get_feature_store() 

#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [18]:
# Convert the dictionary to a JSON string
str_dict = json.dumps(city_dict)

# Replace any existing secret with the new value
secret = secrets.get_secret("SENSOR_LOCATION_JSON")
if secret is not None:
    secret.delete()
    print("Replacing existing SENSOR_LOCATION_JSON")

secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)

Replacing existing SENSOR_LOCATION_JSON
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('SENSOR_LOCATION_JSON', 'PRIVATE')

### <span style="color:#ff5f27;">Create the Feature Groups and insert the DataFrames in them </span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each air quality sensor measurement is uniquely identified by `country`, `street`, and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [19]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

#### Insert the DataFrame into the Feature Group

In [20]:
air_quality_fg.insert(df_aq)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1718620
2025-11-15 11:53:25,691 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1718620


Uploading Dataframe: 100.00% |██████████| Rows 4208/4208 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279155/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758797
         }
       },
       "result": {
         "observed_value": 2.0,
         "element_count": 4208,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T10:53:25.000691Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expect

#### Enter a description for each feature in the Feature Group

In [21]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")
# air_quality_fg.update_feature_description("lag1", "1 day lagged airquality measure")
# air_quality_fg.update_feature_description("lag2", "2 day lagged airquality measure")
# air_quality_fg.update_feature_description("lag3", "3 day lagged airquality measure")


### <span style='color:#ff5f27'> 🌦 Weather Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each weather measurement is uniquely identified by `city` and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [22]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

#### Insert the DataFrame into the Feature Group

In [23]:
# Insert data
weather_fg.insert(weather_df, wait=True)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1718621
2025-11-15 11:54:13,755 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1718621


Uploading Dataframe: 100.00% |██████████| Rows 4292/4292 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279155/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-15 11:54:31,423 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-15 11:54:34,603 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-15 11:56:19,631 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-15 11:56:19,794 INFO: Waiting for log aggregation to finish.
2025-11-15 11:56:28,433 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758798
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 4292,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T10:54:13.000755Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_colu

#### Enter a description for each feature in the Feature Group

In [24]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02: Daily Feature Pipeline 
 </span> 


## <span style="color:#ff5f27;">⏭️ **Exercises:** 
 </span> 

Extra Homework:

  * Try adding a new feature based on a rolling window of 3 days for 'pm25'
      * This is not easy, as forecasting more than 1 day in the future, you won't have the previous 3 days of pm25 measurements.
      * df.set_index("date").rolling(3).mean() is only the start....
  * Parameterize the notebook, so that you can provide the `country`/`street`/`city`/`url`/`csv_file` as parameters. 
      * Hint: this will also require making the secret name (`SENSOR_LOCATION_JSON`), e.g., add the street name as part of the secret name. Then you have to pass that secret name as a parameter when running the operational feature pipeline and batch inference pipelines.
      * After you have done this, collect the street/city/url/csv files for all the sensors in your city or region and you make dashboards for all of the air quality sensors in your city/region. You could even then add a dashboard for your city/region, as done [here for Poland](https://github.com/erno98/ID2223).

Improve this AI System
  * As of mid 2024, there is no API call available to download historical data from the AQIN website. You could improve this system by writing a PR to download the CSV file using Python Selenium and the URL for the sensor.


---